<a href="https://colab.research.google.com/github/kariboter/buffer/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade efficientnet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=37108ac3f38db823f91a6e0d4bc1df013ddb89b99ef3d3d3cbac4e314bcf23f8
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [2]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 22.0 MB/s 


In [3]:
from torchmetrics import Accuracy
accuracy = Accuracy()

In [4]:
import torchvision
import torch
from torch import nn

In [5]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
model = torchvision.models.efficientnet_b0(weights=['EfficientNet_B0_Weights'])
sum([x.numel() for x in model.parameters()])

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

5288548

In [11]:
model.classifier = nn.Linear(1280,10)

In [7]:
len([x.numel() for x in model.parameters()])

213

In [17]:
for i, param in enumerate(model.parameters()):
  if i < 212:
    param.require_grad=True

In [9]:
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3,weight_decay=0.01,amsgrad=True)
criterion = torch.nn.CrossEntropyLoss()

In [18]:
model = model.to('cuda')

In [12]:
import tqdm

In [19]:
for i in range(5):
  train_acc = 0
  val_acc = 0
  curr_train, curr_val = 0,0
  for batch in tqdm.notebook.tqdm(trainloader):
    x, y = batch[0].to('cuda'), batch[1].to('cuda')
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat,y)
    loss.backward()
    optimizer.step()
    curr_train+=accuracy(y_hat.detach().cpu(),y.detach().cpu())
  train_acc=curr_train/len(trainloader)
  with torch.no_grad():
    for batch in testloader:
      x, y = batch[0].to('cuda'), batch[1].to('cuda')
      y_hat  = model(x)
      curr_val+=accuracy(y_hat.detach().cpu(),y.detach().cpu())
  val_acc=curr_val/len(testloader)

  print(f'Iter{i}, train:{train_acc}, val{val_acc}')

  0%|          | 0/782 [00:00<?, ?it/s]

Iter0, train:0.8434702754020691, val0.8161823153495789


  0%|          | 0/782 [00:00<?, ?it/s]

Iter1, train:0.8693054914474487, val0.8200637102127075


  0%|          | 0/782 [00:00<?, ?it/s]

Iter2, train:0.888447105884552, val0.8178741931915283


  0%|          | 0/782 [00:00<?, ?it/s]

Iter3, train:0.903132975101471, val0.8257364630699158


  0%|          | 0/782 [00:00<?, ?it/s]

Iter4, train:0.9173393249511719, val0.8194665312767029


In [ ]:
optimi